# 6. 數據加載、存儲與文件格式#

In [107]:
%pylab
from pandas import Series, DataFrame
import pandas as pd

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


## 讀寫本文格式的數據##

將text轉換為DataFrame的函數，其選項分為:
- 索引
- 類型推斷 和 數據轉換
- 日期解析
- 佚代
- 不規整數據問題

類型推斷(type inference)是最重要的功能之一，不需要指定列的資料型態

In [2]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [6]:
!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [4]:
# read_csv 讀入 csv檔案
df = pd.read_csv('ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# 也可以讀入table，不過需要指定分隔符號
df = pd.read_table('ex1.csv', sep = ',')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# 沒有欄位名稱列的檔案
!type ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [8]:
# 預設會把第一列當作 欄位名稱列
df = pd.read_csv('ex2.csv', )
df

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [9]:
# 標示沒有欄位名稱列
df = pd.read_csv('ex2.csv', header = None)
df

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [14]:
# 自定義 欄位名稱
fields = ['a', 'b', 'c', 'd', 'message']
df = pd.read_csv('ex2.csv', names = fields)
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [18]:
# 可以 使用 index_col 參數，將某一欄設定為DataFrame的索引
fields = ['a', 'b', 'c', 'd', 'message']
df = pd.read_csv('ex2.csv', names = fields, index_col = 'message')
df

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [23]:
# 可以 使用 index_col 參數，將多個欄設定為DataFrame的層次化索引 
!type ex3.csv
df = pd.read_csv('ex3.csv', index_col = ['key1', 'key2'])
df

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [37]:
# 如果不是以固定的分隔符號來分隔字段，可以用 read_table + regex 作為 sep參數
# 由於列名比資料列的數量少，因此read_table推斷第一列應該是DataFrame的索引
# 以不定數量的空白做分隔
!type "ex3 - 1.csv"  
df = pd.read_table('ex3 - 1.csv', sep = '\s+')
df

	A	B	C
aaa  	-0.264  	-1.026  	-0.619
bbb	  0.927	  0.302	  -0.032
ccc          -0.265	   -0.385	    -0.217
	


,A,B,C
aaa,-0.264,-1.026,-0.619
bbb,0.927,0.302,-0.032
ccc,-0.265,-0.385,-0.217


In [39]:
# 讀檔時，可以用 skiprows 來跳過指定的 rows
!type ex4.csv
df = pd.read_csv('ex4.csv', skiprows = [0, 2, 3], index_col = 'message')
df

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who read CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [45]:
# 缺失數據的處理
# read_csv 會自動判斷，然後以NaN標示缺失數據的位置
!type ex5.csv
df = pd.read_csv('ex5.csv', index_col = 'something')
df

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo 


,a,b,c,d,message
something,,,,,
one,1,2,3,4,NaN
two,5,6,NaN,8,world
three,9,10,11,12,foo


In [47]:
# isnull()，判斷元素是否為NaN
df.isnull()

,a,b,c,d,message
something,,,,,
one,False,False,False,False,True
two,False,False,True,False,False
three,False,False,False,False,False


In [48]:
pd.isnull(df)

,a,b,c,d,message
something,,,,,
one,False,False,False,False,True
two,False,False,True,False,False
three,False,False,False,False,False


In [53]:
# na_values 參數可指定用於標示缺失數據的字串
df = pd.read_csv('ex5.csv', index_col = 'something', na_values = ['NULL'])
df

,a,b,c,d,message
something,,,,,
one,1,2,3,4,NaN
two,5,6,NaN,8,world
three,9,10,11,12,foo


In [56]:
# 為各列分別指定不同的 缺失值標示字串
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
df = pd.read_csv('ex5.csv', na_values = sentinels)
df

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11,12,foo


### 逐塊讀取文本文件###

In [57]:
# 設定 nrows參數，設定讀入的列數
!type ex5.csv
df = pd.read_csv('ex5.csv', nrows = 2)
df

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo 


,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world


In [71]:
# 如果要逐塊讀取，則設定chunksize
!type ex5.csv
chunker = pd.read_csv('ex5.csv', chunksize = 2)
chunker

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo 


In [72]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['something'].value_counts(), fill_value = 0)
tot = tot.sort_values(ascending = False)
tot

two      1
three    1
one      1
dtype: float64

## 將數據寫出到文本格式##

In [82]:
!type ex5.csv
df = pd.read_csv('ex5.csv')
df

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo 


,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [83]:
# 以 to_csv() 將數據寫出到一個 以逗號分隔 的檔案中
df.to_csv('ex5-1.csv')
!type "ex5-1.csv"

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo 


In [84]:
# 寫出的時候，可以設定 sep 參數 指定其他的分隔符號
df.to_csv('ex5-1.csv', sep = '|')
!type "ex5-1.csv"

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo 


In [86]:
# 設定 na_rep 參數，以其他的符號 明確地標示 缺失值
df.to_csv('ex5-1.csv', na_rep = 'NULL')
!type "ex5-1.csv"

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo 


In [87]:
# 可以禁止列出 row, column的標籤
# 不輸出index、header
df.to_csv('ex5-1.csv', na_rep = 'NULL', index = False, header = False) 
!type "ex5-1.csv"

one,1,2,3.0,4,NULL
two,5,6,NULL,8,world
three,9,10,11.0,12,foo 


In [88]:
# 不輸出index
df.to_csv('ex5-1.csv', na_rep = 'NULL', index = False) 
!type "ex5-1.csv"

something,a,b,c,d,message
one,1,2,3.0,4,NULL
two,5,6,NULL,8,world
three,9,10,11.0,12,foo 


In [104]:
# 設定 cols 參數，只寫出一部分的欄位
df
df.to_csv("ex5-1.csv", index = False, cols = ['a', 'b', 'c']) # 好像無效呢?
!type "ex5-1.csv"

something,a,b,c,d,message
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo 


In [105]:
# Series 也有to_csv方法
dates = pd.date_range('1/1/2000', periods = 7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [124]:
ts = Series(np.arange(7), index = dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [112]:
# Series物件 也有to_csv方法
ts.to_csv('treseries.csv')
!type "treseries.csv"

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [126]:
# Series類別 也有to_csv方法 (頂層)
Series.to_csv(ts, 'treseries.csv')
!type "treseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [127]:
# 使用 from_csv 將檔案讀入成為 Series
# 有 date欄位，須設定 parse_dates 參數
ts = Series.from_csv('treseries.csv', parse_dates = True)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

## 手工處理 分隔符號 格式##

In [176]:
# 使用 csv.reader 讀取檔案內容，讀入之後，會將每個 line的資料parse成元組
!type ex5.csv

import csv

with open('ex5.csv') as f:
    reader = csv.reader(f) # 將檔案讀入，每 line是一個元組
    lines = list(reader)

for line in lines:
    print(line)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo 
['something', 'a', 'b', 'c', 'd', 'message']
['one', '1', '2', '3', '4', 'NA']
['two', '5', '6', '', '8', 'world']
['three', '9', '10', '11', '12', 'foo ']


In [142]:
# 將資料做一些整理    
header, values = lines[0], lines[1:]

data_dict = {h:v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '5', '9'),
 'b': ('2', '6', '10'),
 'c': ('3', '', '11'),
 'd': ('4', '8', '12'),
 'message': ('NA', 'world', 'foo '),
 'something': ('one', 'two', 'three')}

In [173]:
# 繼承csv.Dialect即可定義出新的格式
class myDialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    
# 用 delimiter 參數可以指定分隔符號    
!type ex6.csv

import csv

with open('ex6.csv') as f: 
    #reader = csv.reader(f, dialect = myDialect)
    reader = csv.reader(f, delimiter = ';')
    lines = list(reader)

for line in lines:
    print(line)

something;a;b;c;d;message
one;1;2;3;4;NA
two;5;6;;8;world
three;9;10;11;12;foo 
['something', 'a', 'b', 'c', 'd', 'message']
['one', '1', '2', '3', '4', 'NA']
['two', '5', '6', '', '8', 'world']
['three', '9', '10', '11', '12', 'foo ']


In [175]:
# 用 csv.writer可以手工輸出分隔符文件
with open("ex6-o.csv", 'w') as f:
    writer = csv.writer(f, delimiter = ';')
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

!type "ex6-o.csv"

one;two;three

1;2;3

4;5;6

7;8;9



## JSON(JavaScript Object Notation)數據##

In [185]:
obj = """
{
"name": "Wes", 
"place_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"}, {"name": "Wei", "age": 25, "pet": "Cisco"}]
}
"""



In [186]:
# 用 json.loads 可將JSON字串還原成 dict物件
import json

result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'place_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 25, 'name': 'Wei', 'pet': 'Cisco'}]}

In [187]:
# JSON物件其實是 dict 物件
type(result)

dict

In [191]:
# 使用索引，可以探及 dict內部的資料
type(result['siblings'][0]['age'])

int

In [192]:
# json.dumps 可將dict物件轉換成 JSON字串
# json字串 和json物件 需區分清楚
# json物件 其實就是 dict
json.dumps(result)

'{"siblings": [{"age": 25, "name": "Scott", "pet": "Zuko"}, {"age": 25, "name": "Wei", "pet": "Cisco"}], "pet": null, "name": "Wes", "place_lived": ["United States", "Spain", "Germany"]}'

In [254]:
result['siblings']

[{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
 {'age': 25, 'name': 'Wei', 'pet': 'Cisco'}]

In [248]:
# 以JSON物件建構DataFrame
df_siblings = DataFrame(result['siblings'], columns = ['age', 'name', 'pet']).T
df_siblings

,0,1
age,25,25
name,Scott,Wei
pet,Zuko,Cisco


In [249]:
# DataFrame有 to_json() 方法，可將DataFrame序列化
siblings_json_string = df_siblings.to_json()
siblings_json_string

'{"0":{"age":25,"name":"Scott","pet":"Zuko"},"1":{"age":25,"name":"Wei","pet":"Cisco"}}'

In [250]:
siblings_json = json.loads(siblings_json_string)
siblings_json

{'0': {'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
 '1': {'age': 25, 'name': 'Wei', 'pet': 'Cisco'}}

In [251]:
# DataFrame有 from_dict() 方法，可反序列化
df_siblings = DataFrame.from_dict(siblings_json)
df_siblings

,0,1
age,25,25
name,Scott,Wei
pet,Zuko,Cisco


## XML和HTML: Web訊息收集##

In [273]:
# 用 lxml, urllib 
from lxml.html import parse 
from urllib.request import urlopen

parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc = parsed.getroot()

In [275]:
# 找出所有的 ancher
links = doc.findall('.//a')
print(len(links))
for link in links: pass 
#    print(link.text_content(),link.get('href')) 

338


In [286]:
tables = doc.findall('.//table')
calls = tables[1]
puts = tables[2]
rows = calls.findall('.//tr')

def _unpack(row, kind = 'td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content()  for val in elts]

_unpack(rows[0], kind = 'th')

['\n                \n                     Strike\n                    \n                        \ue004\n                        \ue002\n                    \n                \n                ∵ Filter\n            ',
 'Contract Name',
 '\n                \n                    Last\n                    \n                        \ue004\n                        \ue002\n                    \n                \n            ',
 '\n                \n                    Bid\n                    \n                        \ue004\n                        \ue002\n                    \n                \n            ',
 '\n                \n                    Ask\n                    \n                        \ue004\n                        \ue002\n                    \n                \n            ',
 '\n                \n                    Change\n                    \n                        \ue004\n                        \ue002\n                    \n                \n            ',
 '\n    

In [287]:
_unpack(rows[1], kind = 'td')

['\n                        \n                            \n                            ✕\n                            [modify]\n                        \n                    ']

In [289]:
from pandas.io.parsers import TextParser

def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind = 'th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names = header).get_chunk()

call_data = parse_options_data(calls)
put_data = parse_options_data(puts)
call_data[:10]

,Strike   ∵ Filter,Contract Name,Last  ,Bid  ,Ask  ,Change  ,%Change  ,Volume  ,Open Interest  ,Implied Volatility  
0,\n \n ...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,\n 70.00\n,\n AAPL151030C00070000\n,48.95,0,0,0,\n \n 0.00%\n ...,4,0,\n 0.00%\n
2,\n 80.00\n,\n AAPL151030C00080000\n,31.05,0,0,0,\n \n 0.00%\n ...,2,0,\n 0.00%\n
3,\n 85.00\n,\n AAPL151030C00085000\n,29.45,0,0,0,\n \n 0.00%\n ...,90,0,\n 0.00%\n
4,\n 89.00\n,\n AAPL151030C00089000\n,22.40,0,0,0,\n \n 0.00%\n ...,9,0,\n 0.00%\n
5,\n 90.00\n,\n AAPL151030C00090000\n,29.10,0,0,0,\n \n 0.00%\n ...,212,0,\n 0.00%\n
6,\n 94.00\n,\n AAPL151030C00094000\n,24.96,0,0,0,\n \n 0.00%\n ...,75,0,\n 0.00%\n
7,\n 95.00\n,\n AAPL151030C00095000\n,20.51,0,0,0,\n \n 0.00%\n ...,139,0,\n 0.00%\n
8,\n 96.00\n,\n AAPL151030C00096000\n,21.91,0,0,0,\n \n 0.00%\n ...,4,0,\n 0.00%\n
9,\n 98.00\n,\n AAPL151030C00098000\n,20.91,0,0,0,\n \n 0.00%\n ...,146,0,\n 0.00%\n


### 利用 lxml.objectify 解析 XML###

In [331]:
data = """
<MTA>
<INDICATOR>
<INDICATOR_SEQ>
373889
</INDICATOR_SEQ>
<AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
<PERIOD_MONTH>12</PERIOD_MONTH>
</INDICATOR>
</MTA>
"""

In [334]:
# 用 objectify.parse 來 parse XML
from lxml import objectify
import io 


# parsed = objectify.parse(open('mta.xml'))
parsed = objectify.parse(io.StringIO(data)) 
root = parsed.getroot()
root

<Element MTA at 0x8267a88>

In [336]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

data

[{'AGENCY_NAME': 'Metro-North Railroad', 'PERIOD_MONTH': 12}]

In [337]:
perf = DataFrame(data)
perf

,AGENCY_NAME,PERIOD_MONTH
0,Metro-North Railroad,12


In [339]:
# StringIO 可以直接讀入一個 XML字串，交給 parser 處理成 DOM
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(io.StringIO(tag)).getroot()
root

<Element a at 0x8608888>

In [340]:
# 讀取 element 的 attribute
root.get('href')

'http://www.google.com'

In [341]:
# 讀取 element的 innerText
root.text

'Google'

## 二進制數據格式##

In [368]:
# pandas物件都有一個 save方法，可以將物件數據以pickle的形式保存到硬碟
df = pd.read_csv('ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [369]:
type(df)

pandas.core.frame.DataFrame

In [370]:
# 輸出 pickle資料到檔案
df.to_pickle('ex1.pickle')
df = None
df

In [375]:
# 讀入 pickle檔案資料成為物件 
df = pickle.load(open('ex1.pickle', 'rb'))
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [376]:
type(df)

pandas.core.frame.DataFrame

In [377]:
pd.loa

<module 'pandas' from 'D:\\Python\\lib\\site-packages\\pandas\\__init__.py'>

### 使用HDF5###
HDF(Hierarchical Data Format)

可以儲存多個數據集並支援元數據

支援多種壓縮器的即時壓縮

Python 有 PyTables 和 h5py 兩個 HDF5的API

In [383]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [384]:
# pandas 有一個 HDFStore 類別，透過 PyTables來存儲pandas物件
# 開一個檔案以便儲存資料
store = pd.HDFStore('mydata.h5') 

# 將物件序列化 儲存到 HDFStore 檔案中
store['obj1'] = df
store['obj1_col'] = df['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[3,5])
/obj1_col            series       (shape->[3])  

In [385]:
store['obj1']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [390]:
store.close()

AttributeError: 'NoneType' object has no attribute 'close'

In [391]:
store = None

In [393]:
# 重新開啟 HDF5檔案並讀回物件
df = None
store = pd.HDFStore('mydata.h5') 
df = store['obj1']
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [394]:
type(df)

pandas.core.frame.DataFrame

In [395]:
df = None
store.close()
store = None

### 讀取 Microsoft Excel文件###

In [400]:
# 使用 ExcelFile 方法
xls_file = pd.ExcelFile('test.xls')
table = xls_file.parse('Sheet1')
table

""


In [401]:
type(table)

pandas.core.frame.DataFrame

## 使用HTML和Web API##

In [413]:
# requests package
import requests, json 
url = 'https://api.twitter.com/1.1/search/tweets.json?q=python%20pandas' # 這個不能這樣用，還需要另外設定複雜的機制，算了
resp = requests.get(url, verify=True)
resp.text

'{"errors":[{"code":215,"message":"Bad Authentication data."}]}'

In [415]:
data = json.loads(resp.text)
data

{'errors': [{'code': 215, 'message': 'Bad Authentication data.'}]}

## 使用數據庫##

In [421]:
# 使用 SQLite3

import sqlite3

# 連接資料庫
con = sqlite3.connect(':memory:')

# 建立資料表
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);
"""
con.execute(query)
con.commit()

# 插入資料
data = [('Atlanta', 'Georgia', 1.25, 6), 
        ('Tallahassee', 'Florida', 2.6, 3), 
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()


# 查詢資料
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [426]:
# cursor.description 包含 欄位資訊
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [428]:
# 用資料庫的資料建立 DataFrame
df = DataFrame(rows, columns = [f[0] for f in cursor.description])
df

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [432]:
# 使用 pandas.io.sql 來讀取資料庫資料並創建 DataFrame
import pandas.io.sql as sql
df = sql.read_frame('select * from test', con)
df

D:\Python\lib\site-packages\IPython\kernel\__main__.py:3: FutureWarning: read_frame is deprecated, use read_sql
  app.launch_new_instance()


,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


### 存取MongoDB中的數據###

In [435]:
import pymongo
con = pymongo.Connection('localhost')
tweets = con.db.tweets

In [ ]:
import requests, json
url = 'http://search.twitter.com/search.json?q=python%20pandas'
data = json.loads(requests.get(url).text)

In [ ]:
for tweet in data['results']:
    tweets.save(tweet)

In [ ]:
cursor = tweets.find({'from_user': 'wesmckinn'})
tweet_fields = ['created_at', 'from_user', 'id', 'text']
result = DataFrame(list(cursor), columns = tweet_fields)